In [ ]:
import pandas as pd
# from dowhy import gcm
import numpy as np
import matplotlib.pyplot as plt
import gc

import os

In [ ]:
node = "lotus"

In [ ]:
#import exp_filtered
data = pd.read_csv('../Datasets/experiments_valid.csv',header=0, index_col=0)
experiments = pd.DataFrame(data)

experiments.tail(10)

In [ ]:
messages = pd.DataFrame()

for index, row in experiments.iterrows():
    data = pd.read_csv('../../Datasets/partialTables/mess_'+node+'_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
    query_mess = pd.DataFrame(data)
    query_mess = query_mess.drop(columns=["start", "end"])

    messages = pd.concat([query_mess, messages])
    del query_mess
    gc.collect()

    if os.path.exists('../../Datasets/partialTables/ihave_'+node+'_'+str(row['experiment'])+'.csv'):
        data = pd.read_csv('../../Datasets/partialTables/ihave_'+node+'_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
        query_ihave = pd.DataFrame(data)
        query_ihave["type"] = '17.0'
        query_ihave = query_ihave.drop(columns=["_measurement"])
        
        messages = pd.concat([query_ihave, messages])
        del query_ihave
        gc.collect()
    
    if os.path.exists('../../Datasets/partialTables/iwant_'+node+'_'+str(row['experiment'])+'.csv'):
        data = pd.read_csv('../../Datasets/partialTables/iwant_'+node+'_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
        query_iwant = pd.DataFrame(data)
        query_iwant["type"] = '16.0'
        query_iwant = query_iwant.drop(columns=["_measurement"])
    
        messages = pd.concat([query_iwant, messages])
        del query_iwant
        gc.collect()
    

messages.head(10)

In [ ]:
# Transpose the table -> type and count
df = messages.drop(columns=['identifier'])
df.reset_index(inplace=True)
df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)
by_time = df.groupby([df['experiment'],df['type'],pd.Grouper(key="_time", freq='5s')])['index'].count().reset_index()


pivoted = pd.pivot_table(by_time,index=["experiment", pd.Grouper(freq="5s", key="_time")], columns="type", values="index", margins=False).fillna(0)
pivoted = pivoted.reset_index().rename_axis(None, axis=1)

pivoted.head(100)

In [ ]:
#Rename
eventsTS = pivoted.rename(columns={2.0:'messageDuplicated', 3.0: 'messageReceived', 11.0: 'graft', 12.0: 'prune', '16.0': 'iwant', '17.0':'ihave'})

# eventsTS.to_csv('./events20s.csv')
# eventsTS.head(500)

In [ ]:
#Import partial tables and append -> for propagation time

publishMessage = pd.DataFrame()
deliverMessage = pd.DataFrame()

for index, row in experiments.iterrows():
    data = pd.read_csv('../../Datasets/partialTables/mess2_'+node+'_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
    query_mess = pd.DataFrame(data)

    data = pd.read_csv('../../Datasets/partialTables/pub_'+node+'_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
    query_pub = pd.DataFrame(data)
    
    deliverMessage = pd.concat([query_mess, deliverMessage])
    publishMessage = pd.concat([publishMessage, query_pub])

    del query_pub
    del query_mess

    gc.collect()

deliverMessage.head(10)


In [ ]:
publishMessage['_time'] = pd.to_datetime(publishMessage["_time"], format='mixed')#.tz_localize(None)
deliverMessage['_time'] = pd.to_datetime(deliverMessage["_time"], format='mixed')#.tz_localize(None)

# def calcAverageTime(publish, received, expTime, parameter):
expTime = experiments
publish = publishMessage
received = deliverMessage

publish = publish[['_time', 'messageID','experiment']]
received = received[['_time', 'messageID', 'experiment']]

joined = publish.merge(received, on=['messageID', 'experiment'])
joined['diff'] = ((joined['_time_y'] - joined['_time_x'])/ pd.Timedelta(microseconds=1)).astype(int)
joined = joined.loc[joined["diff"] >= 0].dropna()
joined.head(10)


In [27]:
df = joined.drop(columns=['_time_x', 'messageID']).rename(columns={'_time_y' : '_time'})

df.reset_index(inplace=True)
df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)
by_time = df.groupby([df['experiment'],pd.Grouper(key="_time", freq='5s')])['diff'].agg('mean').reset_index()
by_time.head(100)

/tmp/ipykernel_14403/842400401.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,diff
0,390,2023-10-04 18:52:00+00:00,1939.820690
1,390,2023-10-04 18:52:20+00:00,1768.231237
2,390,2023-10-04 18:52:40+00:00,1961.133241
3,390,2023-10-04 18:53:00+00:00,1893.468884
4,390,2023-10-04 18:53:20+00:00,1909.643407
...,...,...,...
95,391,2023-10-04 19:30:00+00:00,938.489362
96,391,2023-10-04 19:30:20+00:00,1056.730337
97,391,2023-10-04 19:30:40+00:00,1017.752809
98,391,2023-10-04 19:31:00+00:00,1041.128713


In [28]:
prop = by_time.rename(columns={'diff':'propagationTime'})

prop = prop.merge(eventsTS, on=['experiment', '_time'], how='outer')
prop.head(100)

,experiment,_time,propagationTime,messageDuplicated,messageReceived,graft,prune,iwant,ihave
0,390,2023-10-04 18:52:00+00:00,1939.820690,NaN,NaN,NaN,NaN,NaN,NaN
1,390,2023-10-04 18:52:20+00:00,1768.231237,1503.0,205.0,0.0,0.0,0.0,349.0
2,390,2023-10-04 18:52:40+00:00,1961.133241,3578.0,499.0,0.0,0.0,0.0,324.0
3,390,2023-10-04 18:53:00+00:00,1893.468884,1249.0,177.0,0.0,0.0,0.0,0.0
4,390,2023-10-04 18:53:20+00:00,1909.643407,3787.0,520.0,0.0,0.0,0.0,325.0
...,...,...,...,...,...,...,...,...,...
95,391,2023-10-04 19:30:00+00:00,938.489362,495.0,63.0,0.0,0.0,0.0,40.0
96,391,2023-10-04 19:30:20+00:00,1056.730337,623.0,81.0,0.0,0.0,0.0,38.0
97,391,2023-10-04 19:30:40+00:00,1017.752809,751.0,94.0,0.0,0.0,0.0,69.0
98,391,2023-10-04 19:31:00+00:00,1041.128713,275.0,41.0,0.0,0.0,0.0,41.0


In [ ]:
rpc = pd.DataFrame()

for idx, exec in experiments.iterrows():
    data = pd.read_csv('../../Datasets/partialTables/rpc_'+node+'_'+str(exec['experiment'])+'.csv', header=0,  index_col=0)
    query_rpc = pd.DataFrame(data)
        
    rpc = pd.concat([rpc,query_rpc])

    del query_rpc
    gc.collect()
        

rpc.head(10)

In [ ]:
rpc = rpc.drop(columns=['start', 'end'])
                       
rpc.reset_index(inplace=True)
rpc['_time'] = pd.to_datetime(rpc["_time"], format='mixed', infer_datetime_format=True)
by_time = rpc.groupby([rpc['experiment'],pd.Grouper(key="_time", freq='5s')])['index'].agg('count').reset_index()
# by_time.head(100)

rpc=by_time.rename(columns={'index':'rpc'})

In [ ]:
message=received

message.reset_index(inplace=True)
message['_time'] = pd.to_datetime(message["_time"], format='mixed', infer_datetime_format=True)
by_time = message.groupby([message['experiment'],pd.Grouper(key="_time", freq='5s')])['messageID'].agg('count').reset_index()
message = by_time

message.head(10)

In [ ]:
message = message.rename(columns={'messageID':'count'})
rpc = rpc.rename(columns={'rpc': 'count'})
total_bandwidth = pd.concat([rpc, message])

total_bandwidth.reset_index(inplace=True)
total_bandwidth['_time'] = pd.to_datetime(total_bandwidth["_time"], format='mixed', infer_datetime_format=True)
by_time = total_bandwidth.groupby([total_bandwidth['experiment'],pd.Grouper(key="_time", freq='5s')])['count'].agg('sum').reset_index()
by_time.head(100)

In [ ]:
by_time['totalBandwidth'] = by_time['count']

totalBandwidth = by_time.drop(columns=['count'])
totalBandwidth.head(10)

In [ ]:
#gossip bandwidth
gossipBandwidth = rpc
gossipBandwidth['gossipReceived'] = gossipBandwidth['count']
gossipBandwidth['gossipBandwidth'] = gossipBandwidth['count']/5
gossipBandwidth = gossipBandwidth.drop(columns=['count'])

gossipBandwidth.head(100)

In [ ]:
#MessageBandwidth

messageBandwidth = message
messageBandwidth['messageBandwidth'] = messageBandwidth['count']/5
messageBandwidth = messageBandwidth.drop(columns=['count'])

messageBandwidth.head(10)

In [ ]:
prop['_time'] = pd.to_datetime(prop["_time"], format='mixed', infer_datetime_format=True)
messageBandwidth['_time'] = pd.to_datetime(messageBandwidth["_time"], format='mixed', infer_datetime_format=True)

messBand = prop.merge(messageBandwidth, on=['experiment', '_time'], how='outer').fillna(0)

messBand.head(500)

In [ ]:
messBand['_time'] = pd.to_datetime(messBand["_time"], format='mixed', infer_datetime_format=True)
totalBandwidth['_time'] = pd.to_datetime(totalBandwidth["_time"], format='mixed', infer_datetime_format=True)

totalBand = messBand.merge(totalBandwidth, on=['experiment', '_time'], how='outer').fillna(0)
totalBand['totalBandwidth'] = (totalBand['totalBandwidth']+totalBand['messageDuplicated'])/5

totalBand.head(500)

In [ ]:
totalBand['_time'] = pd.to_datetime(totalBand["_time"], format='mixed', infer_datetime_format=True)
gossipBandwidth['_time'] = pd.to_datetime(totalBandwidth["_time"], format='mixed', infer_datetime_format=True)

gossipBand = totalBand.merge(gossipBandwidth, on=['experiment', '_time'], how='outer').fillna(0)

gossipBand.head(500)

In [ ]:
consDiscovery = gossipBand.rename(columns={'messageDuplicated' : 'messageOverhead'})#.drop(columns=['bandwidth'])
consDiscovery.head(10)

In [ ]:
consFinal = consDiscovery.merge(experiments, on='experiment')
consFinal = consFinal.drop(columns=['_time', 'start', 'end', 'startUnix', 'endUnix', 'parameter'])#, 'originalStart', 'originalEnd']) 

consFinal.to_csv('../consolidated5s_'+node+'.csv')
consFinal.head(10)
